In [1]:
!pip install datasets evaluate sacrebleu jiwer
!pip install accelerate -U
!pip install transformers[torch]
!pip install sentencepiece


In [2]:
import os
import random
import json
import torch
import torch.nn as nn
from functools import partial
from collections import namedtuple
from dataclasses import dataclass, field, asdict
from huggingface_hub import HfApi
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import Trainer, DataCollatorWithPadding, AutoTokenizer, TrainingArguments
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Seq2SeqTrainer
import warnings
warnings.filterwarnings("ignore")



In [ ]:
# Định nghĩa các hằng số
MODEL_NAME = "vinai/bartpho-syllable"  # Tên mô hình pre-trained
MAX_LENGTH = 512  # Độ dài tối đa của chuỗi input/output
BATCH_SIZE = 4  # Kích thước batch
NUM_EPOCHS = 10  # Số epoch (số lần lặp qua toàn bộ dataset trong quá trình huấn luyện)
LEARNING_RATE = 1e-4  # Tốc độ học của mô hình
EVAL_STEPS = 500  # Số bước giữa mỗi lần đánh giá mô hình
LOGGING_STEPS = 500  # Số bước giữa mỗi lần ghi log
SAVE_TOTAL_LIMIT = 2  # Số lượng checkpoint tối đa được lưu

# Tải dataset và sử dụng một tập nhỏ hơn để thử nghiệm

dataset = load_dataset("bmd1905/vi-error-correction-v2")  # Tải dataset sửa lỗi văn bản tiếng Việt
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(100000))  # Chọn 100,000 mẫu từ tập huấn luyện
small_test_dataset = dataset["test"].shuffle(seed=42).select(range(20000))  # Chọn 20,000 mẫu từ tập kiểm tra

# Tải tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)  # Tải tokenizer từ mô hình pre-trained

# Hàm tiền xử lý
def preprocess_function(examples):
    return tokenizer(
        examples["input"],  # Văn bản đầu vào có lỗi
        text_target=examples["output"],  # Văn bản đầu ra đã được sửa lỗi
        max_length=MAX_LENGTH,  # Độ dài tối đa của chuỗi
        truncation=True  # Cắt ngắn các chuỗi dài hơn max_length
    )

# Token hóa datasets
tokenized_train_dataset = small_train_dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)
tokenized_test_dataset = small_test_dataset.map(preprocess_function, batched=True, remove_columns=dataset["test"].column_names)
# Tải mô hình
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)  # Tải mô hình pre-trained
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)  # Tạo data collator để chuẩn bị batch dữ liệu
# Chỉ số để đánh giá
metric = evaluate.load("sacrebleu")  # Tải metric sacrebleu để đánh giá chất lượng mô hình
# Hàm tính toán chỉ số
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)  # Giải mã các dự đoán
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)  # Giải mã các nhãn

    decoded_preds = [pred.strip() for pred in decoded_preds]  # Xóa khoảng trắng thừa
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)  # Tính chỉ số sacrebleu
    return {"sacrebleu": result["score"]}  # Trả về chỉ số sacrebleu


In [4]:
# Tham số huấn luyện
args = Seq2SeqTrainingArguments(
    do_train=True,  # Bật chế độ huấn luyện
    do_eval=True,  # Bật chế độ đánh giá
    output_dir="./vietnamese-fix-v2",  # Thư mục để lưu mô hình
    num_train_epochs=NUM_EPOCHS,  # Số epoch
    learning_rate=LEARNING_RATE,  # Tốc độ học
    per_device_train_batch_size=BATCH_SIZE,  # Kích thước batch cho huấn luyện
    per_device_eval_batch_size=BATCH_SIZE,  # Kích thước batch cho đánh giá
    gradient_accumulation_steps=2,  # Số bước tích lũy gradient
    evaluation_strategy="steps",  # Chiến lược đánh giá theo số bước
    eval_steps=EVAL_STEPS,  # Số bước giữa mỗi lần đánh giá
    save_strategy="steps",  # Chiến lược lưu mô hình theo số bước
    logging_steps=LOGGING_STEPS,  # Số bước giữa mỗi lần ghi log
    save_total_limit=SAVE_TOTAL_LIMIT,  # Số lượng checkpoint tối đa được lưu
    predict_with_generate=True,  # Sử dụng chế độ generate để dự đoán
    fp16=True,  # Sử dụng tính toán FP16 để tăng tốc độ và giảm bộ nhớ
    push_to_hub=False,  # Không đẩy mô hình lên Hugging Face Hub
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,  # Mô hình để huấn luyện
    args=args,  # Tham số huấn luyện
    train_dataset=tokenized_train_dataset,  # Dataset huấn luyện đã được token hóa
    eval_dataset=tokenized_test_dataset,  # Dataset đánh giá đã được token hóa
    data_collator=data_collator,  # Data collator
    tokenizer=tokenizer,  # Tokenizer
    compute_metrics=compute_metrics,  # Hàm tính toán chỉ số
)

# Đánh giá và huấn luyện
trainer.evaluate()  # Đánh giá mô hình trước khi huấn luyện
trainer.train()  # Huấn luyện mô hình
trainer.evaluate()  # Đánh giá mô hình sau khi huấn luyện


Step,Training Loss,Validation Loss,Sacrebleu
500,0.704967,0.298617,33.671179
1000,0.701206,0.295789,33.647689
1500,0.697679,0.294779,34.123458
2000,0.695965,0.294781,33.647689
2500,0.691627,0.293330,33.905188
3000,0.686863,0.291881,34.118698
3500,0.684348,0.293768,34.397942
4000,0.679815,0.289036,34.578703
4500,0.676286,0.288150,33.742747
5000,0.673466,0.286518,33.647689


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-defau

NameError: name 'MODEL_SAVE_PATH' is not defined

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL_NAME = "vinai/bartpho-syllable"
MODEL_SAVE_PATH = "/content/vietnamese-fix-v2" 
MAX_LENGTH = 512

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

tokenizer.save_pretrained(MODEL_SAVE_PATH)
model.save_pretrained(MODEL_SAVE_PATH)


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
